# package

In [1]:
from __future__ import print_function

import argparse
from mxnet.image import image
from mxnet.gluon.data.vision import transforms,CIFAR10,ImageFolderDataset
from mxnet.gluon.data import DataLoader
from mxnet.gluon import Trainer,nn
from mxnet.base import numeric_types
from mxnet.initializer import *
from mxnet import autograd, nd
import mxnet as mx
import mxnet.gluon as gluon

import numpy as np
import math
import os
from mxboard import SummaryWriter
from lsun import LSUN    # Dataset

context = mx.gpu(0)

# Model building
* MLP_G, MLP_D
* DCGAN_G, DCGAN_D
* DCGAN_G_noBN, DCGAN_D_noBN

### MLP_G, MLP_D

In [2]:
class MLP_G(nn.Block):
    def __init__(self, size_img, num_z, num_hidden, num_c):
        super(MLP_G, self).__init__()
        
        self.size_img = size_img
        self.num_c = num_c
        
        with self.name_scope():
            self.base = nn.Sequential()
            self.base.add(nn.Dense(units=num_hidden, in_units=num_z, activation='relu'))
            self.base.add(nn.Dense(units=num_hidden, activation='relu'))
            self.base.add(nn.Dense(units=num_hidden, activation='relu'))
            self.base.add(nn.Dense(units=num_c*size_img*size_img))
        
    def forward(self, input):
        #input = input.reshape((input.shape[0], input.shape[1]))
        output = self.base(input)
        return output.reshape((output.shape[0], self.num_c, self.size_img, self.size_img))

In [3]:
class MLP_D(nn.Block):
    def __init__(self, size_img, num_hidden, num_c):
        super(MLP_D, self).__init__()
        
        #self.size_img = size_img
        #self.num_hidden = num_hidden
        #self.num_c = num_c
        
        with self.name_scope():
            self.base = nn.Sequential()
            self.base.add(nn.Dense(units=num_hidden, in_units=num_c*size_img*size_img, activation='relu'))
            self.base.add(nn.Dense(units=num_hidden, activation='relu'))
            self.base.add(nn.Dense(units=num_hidden, activation='relu'))
            self.base.add(nn.Dense(units=1))
    
    def forward(self, input):
        input = input.reshape((input.shape[0], -1))
        output = self.base(input)
        output = output.mean(axis=1)
        return output

### MLP_G, MLP_D test

In [4]:
imageSize = 64
nz = 100
nc = 3
num_hidden = 64

mlp_G = MLP_G(imageSize, nz, num_hidden, nc)
mlp_G.initialize(mx.init.Xavier(factor_type='in',magnitude=0.01),ctx=context)

In [5]:
input_nd = nd.array(np.ones(shape = (8, 100)), ctx=mx.gpu(0)) # batchsize=8, nz=100
output_nd = mlp_G(input_nd)
print(input_nd.shape)
print(output_nd.shape)

(8, 100)
(8, 3, 64, 64)


In [6]:
mlp_D = MLP_D(imageSize, num_hidden, nc)
mlp_D.initialize(mx.init.Xavier(factor_type='in',magnitude=0.01),ctx=context)

In [7]:
mlp_d_input_nd = output_nd
mlp_d_output_nd = mlp_D(mlp_d_input_nd)
print(mlp_d_input_nd.shape)
print(mlp_d_output_nd.shape)

(8, 3, 64, 64)
(8,)


In [8]:
mlp_d_output_nd


[5.7439184e-12 5.7439184e-12 5.7439184e-12 5.7439184e-12 5.7439184e-12
 5.7439184e-12 5.7439184e-12 5.7439184e-12]
<NDArray 8 @gpu(0)>

# Custom init of G, D

In [ ]:
# custom weights initialization called on netG and netD
def weights_init(layers):
    for layer in layers:
        classname = layer.__class__.__name__
        if classname.find('Conv') != -1:
            layer.weight.set_data(mx.ndarray.random.normal(0.0,0.02,shape=layer.weight.data().shape))
        elif classname.find('BatchNorm') != -1:
            layer.gamma.set_data(mx.ndarray.random.normal(1.0, 0.02,shape=layer.gamma.data().shape))
            layer.beta.set_data(mx.ndarray.zeros(shape=layer.beta.data().shape))

In [ ]:
weights_init(mpl_G.base)

# LSUN Dataset and DataLoader

In [ ]:
dataroot = '/home/rich/Desktop/Datasets/lsun'
dataset = LSUN(root=dataroot, classes=['bedroom_train'],
               transform=transforms.Compose([
                   transforms.Resize(imageSize, keep_ratio=True, interpolation=3),
                   transforms.CenterCrop(imageSize),
                   transforms.ToTensor(),
                   transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),]))

In [ ]:
assert dataset
batch_size = 8
dataloader = DataLoader(dataset, batch_size=batch_size,
                        shuffle=True, num_workers=4)

# Train Process of MLP_G, MLP_D

In [ ]:
trainer_G = Trainer(mlp_G.collect_params(),optimizer='adam',optimizer_params={'learning_rate': opt.lrG, 'beta1': opt.beta1, 'beta2': 0.999})
trainer_D = Trainer(mlp_D.collect_params(),optimizer='adam',optimizer_params={'learning_rate': opt.lrD,'beta1': opt.beta1,'beta2':0.999})